<h1>Proyecto, twitter<span class="tocSkip"></span></h1> 

>**Maestría en Analítica de Datos**  
>**Facultad de Ingeniería y Ciencias Básicas.**  
>**Universidad Central  2021 - III**  
>**Integrantes del trabajo:**  
- Maria Alejandra Castillo Pabon
- David Alejandro Ballesteros Díaz


# Librerias y Parametros de Conexión

In [1]:
# !pip install tweepy
# !pip install selenium
# !pip install pyvis
import re
import csv
import pandas as pd
from getpass import getpass
import numpy as np
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from msedge.selenium_tools import Edge, EdgeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import tweepy
import json
from datetime import date
from datetime import datetime
!pip install nltk
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import matplotlib.pyplot as plt
from wordcloud import WordCloud
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
!pip install Goslate
import goslate
from pyvis.network import Network
import seaborn as sns
from textblob import TextBlob
import plotly.graph_objects as go


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\malej\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ModuleNotFoundError: No module named 'wordcloud'

# Funciones personalizadas (16)
## Selenium Bot (7)

### Activar Bot

In [2]:
def controlador ():
    options = EdgeOptions()
    options.use_chromium = True
    driver = Edge(options=options)
    return(driver)

### Ir a una url

In [3]:
def ir_pagina (driver,url):
    driver.get(url)

### Iniciar sesion en twitter

In [4]:
def iniciar_sesion(driver,url,user,my_password):
    ir_pagina(driver,url)
    username = driver.find_element_by_xpath('//input[@name="session[username_or_email]"]')
    username.send_keys(user)
    password = driver.find_element_by_xpath('//input[@name="session[password]"]')
    password.send_keys(my_password)
    password.send_keys(Keys.RETURN)
    return('logueado')  

### Ir a los tweet y respuestas de una persona

In [5]:
def pagina_tweet (driver,nombre):
    url='https://twitter.com/'+nombre+'/with_replies'
    ir_pagina(driver,url)
    driver.execute_script("document.body.style.zoom='50%'")
    return ('conectado a la página '+url)

### Obtener tweet

In [6]:
def obtener_tweet(card):

    username = card.find_element_by_xpath('.//span').text
    try:
        handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + responding
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text
#    link= card.find_elements_by_css_selector('.css-4rbku5.css-18t94o4.css-901oao.r-9ilb82.r-1loqt21.r-1q142lx.r-1qd0xha.r-a023e6.r-16dba41.r-rjixqe.r-bcqeeo.r-3s2u2q.r-qvutc0')[0].get_attribute('href')  
#    link= card.find_elements_by_css_selector('.css-4rbku5.css-18t94o4.css-901oao.r-9ilb82.r-1loqt21.r-1q142lx.r-37j5jr.r-a023e6.r-16dba41.r-rjixqe.r-bcqeeo.r-3s2u2q.r-qvutc0')[0].get_attribute('href')  
    link= card.find_elements_by_css_selector('.css-4rbku5.css-18t94o4.css-901oao.r-14j79pv.r-1loqt21.r-1q142lx.r-37j5jr.r-a023e6.r-16dba41.r-rjixqe.r-bcqeeo.r-3s2u2q.r-qvutc0')[0].get_attribute('href')  
    emoji_tags = card.find_elements_by_xpath('.//img[contains(@src, "emoji")]')
    emoji_list = []
    for tag in emoji_tags:
        filename = tag.get_attribute('src')
        try:
            emoji = chr(int(re.search(r'svg\/([a-z0-9]+)\.svg', filename).group(1), base=16))
        except AttributeError:
            continue
        if emoji:
            emoji_list.append(emoji)
    emojis = ' '.join(emoji_list)
    
    tweet = (username, handle, postdate, text, emojis, reply_cnt, retweet_cnt, like_cnt,link)
    return tweet

### Extracción de los Tweets

In [27]:
def inf_tweets (driver,fecha_fin = "01-01-2001"):
    data = []
    tweet_ids = set()
    last_position = driver.execute_script("return window.pageYOffset;")
    scrolling = True
    while scrolling:
        page_cards = driver.find_elements_by_xpath('//articulo[@data-testid="tweet"]')
        for card in page_cards[-15:]:
            tweet = obtener_tweet(card)
            if tweet is None:
                pass              
            else:
                if 1 ==1 :
                    tweet_id = ''.join(tweet)
                    if tweet_id not in tweet_ids:
                        tweet_ids.add(tweet_id)
                        data.append(tweet)
                else:
                    scrolling=False
        scroll_attempt = 0
        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(2)
            curr_position = driver.execute_script("return window.pageYOffset;")
            if last_position == curr_position:
                scroll_attempt += 1

                if scroll_attempt >= 3:
                    scrolling = False
                    break
                else:
                    sleep(2) # attempt another scroll
            else:
                last_position = curr_position
                break
    print("fin extracción")
    return (data)

### Guardar Tweets

In [23]:
def guardar_tweets (nombre_archivo,lista):
    with open(nombre_archivo, 'w', newline='', encoding="utf-8") as f:
        header = ['nombre', 'usuario', 'fecha', 'texto', 'emoticon', 'cantidad_comentarios', 'cantidad_likes', 'Cantidad_Retweets']
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(lista)
    return ('Se guardo el arhivo '+nombre_archivo+' con '+str(len(lista))+' registros')

## Selenium

In [16]:
usuario='@alejacp4'
clave = getpass()
url='https://www.twitter.com/login'


········


In [8]:
driver=controlador()

In [9]:
## petrogustavo (david)
## MariaFdaCabal  (david)
## ingrodolfohdez (aleja)

pagina_tweet(driver,'MariaFdaCabal')

'conectado a la página https://twitter.com/MariaFdaCabal/with_replies'

In [28]:
## from:sergio_fajardo since:2021-07-01 until:2021-08-29
## from:sergio_fajardo since:2021-01-01 until:2021-07-01
## from:ingrodolfohdez since:2021-01-01 until:2021-08-29
tweets=inf_tweets(driver,"01-06-2000") 
guardar_tweets ("prueba.csv",tweets)
#guardar_tweets ("pruebapetrolink.csv",tweets)

fin extracción


'Se guardo el arhivo prueba.csv con 0 registros'

In [ ]:
pagina_tweet(driver,'ingrodolfohdez')

In [ ]:
tweets=inf_tweets(driver,"01-06-2020")
guardar_tweets ("ingrodolfohdez2708.csv",tweets)

## Carga de Información

In [ ]:
df_tweets= pd.read_csv('tweets_sergio.csv')
df_tweets.head(3)

## Tweepy (2)
### Información General

In [21]:
def inf_general(nombre):
    columnas = ['fecha_reporte','id', 'nombre', 'usuario','descripcion','cantidad_seguidores','cantidad_sigue','fecha_creacion']
    df_inf_general = pd.DataFrame(columns=columnas)
    data = api.get_user("@"+nombre)
    id_fajardo=data._json ["id"]
    sigue=data._json ["friends_count"]
    df_inf_general=df_inf_general.append({'fecha_reporte':date.today(),
                                         'id':id_fajardo,
                                         'nombre':data._json ["name"],
                                         'usuario':data._json ["screen_name"],
                                         'descripcion':data._json ["description"],
                                         'cantidad_seguidores':data._json ["followers_count"],
                                         'cantidad_sigue':sigue,
                                         'fecha_creacion':data._json ["created_at"]},ignore_index=True)
    return(df_inf_general)

### Personas que sigue

In [22]:
def sigue_a(numero_id,cantidad):
    columnas = ['id', 'nombre', 'usuario','descripcion','cantidad_seguidores','cantidad_sigue','fecha_creacion']
    df_sigue = pd.DataFrame(columns=columnas)
    for user in tweepy.Cursor(api.friends, id=numero_id).items(cantidad):
        df_sigue=df_sigue.append({'id':user._json ["id"],
                                  'nombre':user._json ["name"],
                                  'usuario':user._json ["screen_name"],
                                  'descripcion':user._json ["description"],
                                  'cantidad_seguidores':user._json ["followers_count"],
                                  'cantidad_sigue':user._json ["friends_count"],
                                  'fecha_creacion':user._json ["created_at"]},ignore_index=True)
    return(df_sigue)   

## Análisis (3)

### Limpieza tweets

In [23]:
def limpiar(texto):
    nuevo_texto = texto
    if nuevo_texto is None:
        pass
    else:
        try:
            nuevo_texto = texto.lower()
        except:
            pass
        nuevo_texto = re.sub('http\S+', ' ', nuevo_texto)
        regex = '[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\-\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~\\¡\\¿]'
        nuevo_texto = re.sub(regex , ' ', nuevo_texto)
        nuevo_texto = re.sub("\d+", ' ', nuevo_texto)
        nuevo_texto = re.sub("\\s+", ' ', nuevo_texto)
        nuevo_texto=nuevo_texto.replace("replying to", "")
        nuevo_texto=nuevo_texto.replace("Replying to", "")
        nuevo_texto=nuevo_texto.replace("sergio fajardo", "")
    return(nuevo_texto)

### Traducción Textos

In [24]:
def traducir (texto):
    gs = goslate.Goslate()
    texto=gs.translate(texto, 'en')
    return texto

### Clasificación Sentimientos

In [25]:
def clasificacion (df):
    condiciones = [(df['Sentimiento']< 0),
                   (df['Sentimiento'] <= 0.4),
                   (df['Sentimiento'] <= 1)]
    rangos=np.array(("Negativo", "Neutro", "Positivo"), dtype="str")
    df["sentimiento1"] = np.select(condiciones, rangos)
    return (df)

## Visualización (3)

### LLuvia de palabras

In [26]:
def lluvia(df,texto):
    stop_words = list(stopwords.words('spanish'))
    stop_words.extend(("sergio", "fajardo", "sergio fajardo","hoy","sergiofajardo"))
    df=df.dropna()
    text = ' '.join(df) 
    wordcloud = WordCloud(width=1024, height=800, stopwords=stop_words, background_color="white", min_font_size=14).generate(text) 
    plt.figure(figsize= (8, 8), facecolor=None)
    plt.suptitle(texto,size=20) 
    plt.imshow(wordcloud)
    plt.axis('off') 
    plt.tight_layout(pad = 0) 
    return (plt.show())

### Red

In [27]:
def grafro(df, usuario,cantidad,nombre,texto):
    ids=[nombre for nombre in df[usuario]]
    valor=[valor for valor in df[cantidad]]
    bonds=[[0,x+1] for x in range (len (df))]
    titulo=[valor for valor in df[nombre]]

    g = Network("850px", "95%",notebook=True,font_color='white',heading=texto)
    for x in range(len(ids)+1):
        if x == 0:
            g.add_node(x,label='Sergio Fajardo',value='1587923',physics=True,size=30,title='Sergio Fajardo\n->Seguidores: '+ format(1587923,',d'),color='#0044ff',)            
        else:
            if valor[x-1] > 1000000:
                g.add_node(x,label=' ',value=valor[x-1],physics=True,size=30,title=titulo[x-1]+'\n @'+ids[x-1]+'\n->Seguidores: '+ format(valor[x-1],',d'),color='#ff0077')        

            g.add_node(x,label=' ',value=valor[x-1],physics=True,size=30,title=titulo[x-1]+'\n @'+ids[x-1]+'\n->Seguidores: '+ format(valor[x-1],',d'))

    g.add_edges(bonds)
    return(g.show('comunidad.html'))

### Pie

In [28]:
def pie (df):
    df=df[['sentimiento1','Sentimiento']].groupby(['sentimiento1'],as_index=False).count()
    fig = go.Figure(data=[go.Pie(labels=df['sentimiento1'], values=df['Sentimiento'], hole=.6)])
    return(fig.show())

# Parámetros
## Tweepy

In [29]:
consumer_key = "SlwdATPasmyIhHrBUiQ0SyH6f"
consumer_secret = "38DRuXjkv7lb7C2Ufqw5JupKjUfMh5U0HS7HoSRfjxnhCMy9eJ"
access_token = "294848614-Sf5ulXpdIfrMrabSgkDJAvXbN3GhWN0TTDlT3JJU"
access_token_secret = "EntjBRbv9QGetACHmP5iWDv5qQTBpDJIDvSBchvXMIaiU"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth,
                 wait_on_rate_limit=True,
                 wait_on_rate_limit_notify=True)

## Centrar Gráficos

In [ ]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

# Extracción de Información Sergio Fajardo

## Información General

In [ ]:
inf_general('sergio_fajardo')

## Las personas que sigue

In [ ]:
sigue_a(25185308,415).head(5)

In [ ]:
#df_sigue.to_csv('seguidor.csv', header=True, index=False)

# Twitters publicados por Sergio Fajardo

## Proceso de scraping de twitter
![selinium](https://raw.githubusercontent.com/Daviddalejandro/Visualizacion/2eeaa18da1fcdf7a44c0d549fcb19f94e34c5182/imagenes/Selinium.gif "selinium")

# Tweets mas comentados

In [ ]:
links=[link for link in df_tweets.sort_values('cantidad_comentarios',ascending=False).head(3).link]
texto=[texto for texto in df_tweets.sort_values('cantidad_comentarios',ascending=False).head(3).texto]
for x in range(len(links)):
    print('Link '+str([x+1])+' '+links[x],'\n')
    print(texto[x],'\n')

# Extracción de Información de tweets mas comentados

## Link 1

In [ ]:
driver=controlador()
ir_pagina(driver,links[0])
tweets=inf_tweets(driver)
guardar_tweets ("link1.csv",tweets)

## Link 2

In [ ]:
#driver=controlador()
ir_pagina(driver,links[1])
tweets=inf_tweets(driver)
guardar_tweets ("link2.csv",tweets)

## Link 3

In [ ]:
#driver=controlador()
ir_pagina(driver,links[2])
tweets=inf_tweets(driver)
guardar_tweets ("link3.csv",tweets)

# Tendencia política de Sergio Fajardo en twitter 

## Carga de Información

In [ ]:
df_tweets= pd.read_csv('tweets_sergio.csv')
df_tweets = df_tweets.where(pd.notnull(df_tweets), None)
df_link1=pd.read_csv('df_link1.csv')
df_link1 = df_link1.where(pd.notnull(df_link1), None)
df_link2=pd.read_csv('df_link2.csv')
df_link2 = df_link2.where(pd.notnull(df_link2), None)
df_link3=pd.read_csv('df_link3.csv')                   
df_link3 = df_link3.where(pd.notnull(df_link3), None)
df_sigue= pd.read_csv('df_sigue.csv')
df_sigue = df_sigue.where(pd.notnull(df_sigue), None)

## Tranformación de los datos

In [ ]:
sid= SentimentIntensityAnalyzer()
df_tweets['texto_tokenizado'] = df_tweets['texto'].apply(limpiar)

df_link1['texto_tokenizado'] = df_link1['texto'].apply(limpiar)
#df_link1['traduccion']=df_link1['texto_tokenizado'].apply(traducir)
df_link1['Sentimiento']=df_link1['traduccion'].apply(lambda i: sid.polarity_scores(str(i))['compound'])
df_link1=clasificacion(df_link1)

#df_link2['texto_tokenizado'] = df_link2['texto'].apply(limpiar)
#df_link2['traduccion']=df_link2['texto_tokenizado'].apply(traducir)
df_link2['Sentimiento']=df_link2['traduccion'].apply(lambda i: sid.polarity_scores(str(i))['compound'])
df_link2=clasificacion(df_link2)

#df_link3['texto_tokenizado'] = df_link3['texto'].apply(limpiar)
#df_link3['traduccion']=df_link3['texto_tokenizado'].apply(traducir)
df_link3['Sentimiento']=df_link3['traduccion'].apply(lambda i: sid.polarity_scores(str(i))['compound'])
df_link3=clasificacion(df_link3)

## Visualización

In [ ]:
lluvia(df_tweets['texto_tokenizado'],"Palabras más usuadas por Fajardo en sus tweets")
grafro(df_sigue, 'usuario','cantidad_seguidores','nombre','A quién sigue Sergio Fajardo?')

## Link 1

In [ ]:
print(texto[0])
lluvia(df_link1['texto_tokenizado'],'')
pie(df_link1)

## Link 2

In [ ]:
print(texto[1])
lluvia(df_link2['texto_tokenizado'],'')
pie(df_link2)

## Link 3

In [ ]:
print(texto[2])
lluvia(df_link3['texto_tokenizado'],'')
pie(df_link3)